[View in Colaboratory](https://colab.research.google.com/github/pasumarthi/DeepLearning/blob/master/CIFAR10_v12.ipynb)

In [1]:
#@title Default title text
#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam,SGD

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
import os
print( os.getcwd() )
print( os.listdir() )

/content
['datalab', '.config', '.local', '.keras', '.forever', '.ipython', '.rnd', '.cache']


In [0]:
batch_size = 64
num_classes = 10
epochs = 50
l = 20
num_filter = 12
compression = 0.6
dropout_rate = 0.3
name='CIFAR10_Batch4'
data_augmentation = True

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#print(x_train.shape[1:])

170500096/170498071 [==============================] - 33s 0us/step
32 32 3


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = MaxPooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.3
weight_decay = 1e-4
l = 12
input = Input(shape=(img_height, img_width, channel,))
#First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
First_Conv2D = Conv2D(num_filter*2, (3,3),use_bias=False ,padding='same')(input)
Second_Con2D =Conv2D(num_filter*4,(7,1),use_bias=False ,padding='same')(First_Conv2D)
Third_Con2D=Conv2D(num_filter, (1,7), use_bias = False, padding = 'same')(Second_Con2D)
Forth_Con2D=Conv2D(num_filter, (1,1), use_bias = False, padding = 'same')(Third_Con2D)

First_Block = add_denseblock(Forth_Con2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [40]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_148 (Conv2D)             (None, 32, 32, 24)   648         input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_149 (Conv2D)             (None, 32, 32, 48)   8064        conv2d_148[0][0]                 
__________________________________________________________________________________________________
conv2d_150 (Conv2D)             (None, 32, 32, 12)   4032        conv2d_149[0][0]                 
__________________________________________________________________________________________________
conv2d_151

In [0]:
#@title Default title text
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
lr_schedule = [10, 20, 30] # epoch_step
def schedule(epoch_idx):
    if (epoch_idx + 1) < lr_schedule[0]:
        return 0.1
    elif (epoch_idx + 1) < lr_schedule[1]:
        return 0.02 # lr_decay_ratio = 0.2
    elif (epoch_idx + 1) < lr_schedule[2]:
        return 0.004
    return 0.0008

In [0]:
import keras.callbacks as callbacks
from keras.callbacks import LearningRateScheduler
save_c = callbacks.ModelCheckpoint(name + '{epoch:02d}-{val_acc:.3f}.hdf5', monitor="val_acc", save_best_only=True,save_weights_only=True,mode='max',period=1)
lrs = LearningRateScheduler(schedule=schedule)

In [44]:
from keras.preprocessing.image import ImageDataGenerator
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
    

Using real-time data augmentation.


In [45]:
#train_datagen=datagen.fit(x_train, augment=True)
#validate_datagen=datagen.fit(x_test, augment=True)
# compute quantities required for featurewise normalization
datagen.fit(x_train, augment=True)
datagen.fit(x_test, augment=True)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size= 128),
          callbacks=[save_c,lrs],
          epochs=45,
          verbose=1,
          validation_data=datagen.flow(x_test, y_test, batch_size= 64))

Epoch 1/45
 95/391 [======>.......................] - ETA: 1:24:02 - loss: 4.0318 - acc: 0.1027

In [59]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 16s 2ms/step
Test loss: 1.2412913593292236
Test accuracy: 0.5969


In [60]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model45.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model45.h5')

In [0]:
# Saving using PyDrive to google drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'Suresh_ASSIGNMENT4B_DNST_model.h5'})
uploaded.SetContentFile('Suresh_ASSIGNMENT4B_DNST_model.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
#downloaded = drive.CreateFile({'id': uploaded.get('id')})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))